In [7]:
import pandas as pd
data=pd.read_csv('C:\\Users\\DIC\\ecsvfile.txt', sep=" ",names=["Catalina_Surveys_ID","ID","RA_(J2000)"," Dec ","V_(mag)","Period_(days)","Amplitude","Number_Obs","Var_Type"])
data1=pd.read_csv('C:\\Users\\DIC\\ALLVar', sep=",",header=None ,names=["ID","MJD","Mag","Magerr","RA","Dec"])
result = data[['ID','Var_Type']].merge(data1[['ID','MJD','Mag']], on = 'ID',how = 'left')
result1=result.round({'Mag': 6})
result2=result1.dropna(axis=0)
result2.to_csv('merged.txt', encoding='utf-8',index=False)


In [2]:
import numpy as np
import pandas as pd 


In [8]:
(result1.loc[result1['Var_Type'] == 5]).to_csv('vartype_5_data.txt',encoding='utf-8',index=False)


In [10]:
import pandas as pd
data22=pd.read_csv('C:\\Users\\DIC\\vartype_5_data.txt')
data22[data22.isnull().any(axis=1)]
result5=data22.dropna(axis=0)

result5.to_csv('variable_without.txt',encoding='utf-8',index=False)


In [11]:
import pandas as pd
data2=pd.read_csv('C:\\Users\\DIC\\variable_without.txt')
data3 = data2.groupby('ID')
data3.apply(lambda x: x.to_csv('ID' + str(x.name) + '.txt',index=False,header=None))

""


In [1]:
import glob
import numpy as np

#l = [pd.read_csv(filename) for filename in glob.glob("C:\\Users\\Shreya choudhary\\variable type\\*.txt")]

file_list = glob.glob("C:\\Users\\Shreya Choudhary\\var13_sample" + "\\*.txt") #Get folder path containing text files
abc = pd.DataFrame([])
training_data=[]
#print(file_list)
d=0
for file_path in file_list:
    ID,vartype,MJD,Mag=np.loadtxt(file_path, unpack=True, delimiter=',',dtype=None)
    c=[]
    e=[]
    for i in range(0,len(Mag)):
        for j in range(i+1,len(Mag)):
            c.append(Mag[i]-Mag[j])
    #print(c)   
    for i in range(0,len(MJD)):
        for j in range(i+1,len(MJD)):
            e.append(MJD[j]-MJD[i])
    #print(e)
    s1 = pd.Series(c, name='dm_y')

    s2 = pd.Series(e,name='dt_x')



#Assigning bins to all dmdt values

    bins=[-8,-5,-3,-2.5,-2,-1.5,-1,-0.5,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.5,1,1.5,2,2.5,3,5,8]
    bins1=[1/145,2/145,3/145,4/145,1/25,2/25,3/25,1.5,2.5,3.5,4.5,5.5,7,10,20,30,60,90,120,240,600,960,2000,4000] #bins of x axis
    return_bins_of_s2= np.digitize(s2, bins1) # np.digitize gives the bin number to which a particular value of dm, dt belongs to.
    return_bins_of_s1 = np.digitize(s1, bins)
    ybins = pd.Series(return_bins_of_s1, name='ybin')
    xbins = pd.Series(return_bins_of_s2, name='xbin')

    dmdt=pd.concat([s1, s2,ybins,xbins], axis=1)
    #print(dmdt)


# exclude all the values from dataframe dmdt that are outside the bin boundries i.e. bin number 0, 23 and 24
    dmdt=dmdt[dmdt.ybin != 23]
    dmdt=dmdt[dmdt.xbin != 0]
    dmdt=dmdt[dmdt.ybin != 0]
    dmdt=dmdt[dmdt.xbin != 24]


    dmdt['bins_xaxis']=pd.cut(dmdt['dt_x'],bins1) #pd.cut gives range of bin for all dt_x values 
    dmdt['bins_yaxis']=pd.cut(dmdt['dm_y'],bins)


    dmdt['merged_dmdt']=  dmdt.dm_y.map(str) + ',' + dmdt.dt_x.map(str)
    dmdt['dmdt_bin_number']=  dmdt.ybin.map(str) + ',' + dmdt.xbin.map(str)

    dmdt.dropna() # drops all the values with NaN in dataframe dmdt
    #print(dmdt)
 


    dmdtpairs=len(dmdt.dm_y) # number of dmdtpairs for a light curve of length n that falls under the given bin range
    #print(dmdtpairs)


#grouping and counting all the values that fall in all individual bins
    df=dmdt.groupby('dmdt_bin_number',sort=False).merged_dmdt.agg(['count']) #counts how many merged_dmdt benlongs to each 'dmdt_bin_number' range
    #print(df)
    df.to_csv('count1.txt',encoding='utf-8')

    data_count=pd.read_csv('C:\\Users\\Shreya Choudhary\\count1.txt')

    ldf=dmdt.set_index('dmdt_bin_number').join(data_count.set_index('dmdt_bin_number'))

    abcc=ldf.dropna()
    final=abcc.drop_duplicates(['ybin','xbin']) # drops all the rows where both of values of ybin and xbin are duplicate
    final1=final.drop(['bins_xaxis','bins_yaxis','merged_dmdt'],axis=1)
    #print(final1)
    final1.to_csv('final_table11.txt',encoding='utf-8',header=None,index=False)

    
    dm,dt,xbin,ybin,count=np.loadtxt('final_table11.txt',unpack=True,delimiter=',',dtype=int)

# assignig intensity values to each bin. Bins are in the form of 23x24 numpy array
    
    #array1=[]
    #label1=[1,0] #for 2nd variavle: label2=[0,1]
    aa = np.zeros(shape=(22,23))
    for gg in range(0,len(xbin)):
                    
        l1=xbin[gg]
        m1=ybin[gg]
        o1=count[gg]
        i1=(255 * o1)/(dmdtpairs + 0.99999)
        #print(i1)
        aa[l1-1,m1-1]=i1
    #print (aa)
    aa.flatten()
    training_data.append(aa)
abc = pd.DataFrame({'x_train':training_data})
print (abc)   
      
   


NameError: name 'pd' is not defined

In [71]:
abc['y_train']='0'
print(abc)

                                             x_train y_train
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...       0
1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...       0
2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...       0
3  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...       0


In [76]:
print (abc.x_train[0])

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00

In [67]:
print (abc.iloc[2])
a=abc.iloc[1]
print(a)
abc.iloc[1].to_csv('pandas12.txt', header=None, index=None)


col    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
Name: 2, dtype: object
col    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
Name: 1, dtype: object


In [20]:
import pandas as pd

df_1 = pd.DataFrame(training_data, index=df_1.index) 
#df_1['a_'] = pd.Series(training_data, index=df_1.index)
print(df_1)

ValueError: Must pass 2-d input

In [12]:
array.shape()

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
 
    import seaborn as sns
    sns.set()
    image11=sns.heatmap(aa,xticklabels=False, yticklabels=False,vmin=0, vmax=255,cbar=False,cmap='jet')
#image11=sns.heatmap(aa,xticklabels=False, yticklabels=False)
#vmin=0, vmax=255,
#cmap='RdYlBu'// colormap of heatmap
#sns_plot.figure.savefig("output.png")
    
    image11.figure.savefig("C:\\Users\\Shreya Choudhary\\images_sv13"+"\\Var13_fig%d.png" %d)
    d+=1